In [ ]:
########CELLA 1###############
from google.colab import drive
import zipfile
import os

# 1. Monta Google Drive
drive.mount('/content/drive')

# 2. Configurazione Percorsi
# Assumiamo che il file sia nella root del tuo Drive.
# Se è in una sottocartella, modifica in: '/content/drive/MyDrive/NOME_CARTELLA/dataset.zip'
zip_path = '/content/drive/MyDrive/dataset.zip'
extract_to = '/content/dataset_unzipped'

# 3. Estrazione
if os.path.exists(zip_path):
    print(f"Trovato {zip_path}. Inizio estrazione...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print("✅ Estrazione completata!")
else:
    print(f"❌ ERRORE: Non trovo il file '{zip_path}'.")
    print("Controlla di averlo caricato su Drive e che il nome sia esattamente 'dataset.zip' (tutto minuscolo).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Trovato /content/drive/MyDrive/dataset.zip. Inizio estrazione...
✅ Estrazione completata!


In [ ]:
############CELLA 2#################
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class BookCoverDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, class_to_idx=None):
        """
        Args:
            csv_file (string): Percorso al file CSV.
            root_dir (string): Directory che contiene le immagini (224x224).
            transform (callable, optional): Trasformazioni (Tensor, Normalize).
        """
        # Lettura CSV con i parametri corretti scoperti prima (sep=; encoding=ISO...)
        self.df = pd.read_csv(csv_file, sep=';', encoding='ISO-8859-1', header=0, on_bad_lines='warn')

        self.root_dir = root_dir
        self.transform = transform

        # Ordina le classi per garantire coerenza
        self.classes = sorted(self.df['Category'].unique())

        # Mappa Stringa -> Intero
        if class_to_idx is None:
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        else:
            self.class_to_idx = class_to_idx

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Recupera nome file e costruisce percorso
        img_name = str(self.df.iloc[idx]['Filename'])
        img_path = os.path.join(self.root_dir, img_name)

        # Caricamento Immagine
        try:
            image = Image.open(img_path).convert('RGB')
        except (OSError, FileNotFoundError):
            # Gestione immagine mancante (crea immagine nera)
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        # Label
        label_str = self.df.iloc[idx]['Category']
        label = self.class_to_idx[label_str]

        # Trasformazioni
        if self.transform:
            image = self.transform(image)

        return image, label

# --- CONFIGURAZIONE TRASFORMAZIONI ---
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        # 1. RandomResizedCrop: Taglia pezzi casuali e li riporta a 224x224.
        # Costringe la rete a guardare i dettagli. scale=(0.8, 1.0) significa che prende almeno l'80% dell'immagine.
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),

        transforms.RandomHorizontalFlip(),

        # 2. ColorJitter: Variazione di luminosità e contrasto (fondamentale per scansioni diverse)
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

        transforms.RandomRotation(15),

        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)), # Nel validation NON facciamo augmentation
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [ ]:
#################CELLA 3####################
# Cerchiamo i percorsi corretti navigando tra le cartelle estratte
base_search_path = '/content/dataset_unzipped'
csv_path = None
img_dir = None

print("🔍 Scansione cartelle in corso...")

for root, dirs, files in os.walk(base_search_path):
    # Cerchiamo il CSV di training
    if "book30-listing-train.csv" in files:
        csv_path = os.path.join(root, "book30-listing-train.csv")
        print(f"   -> CSV Trovato: {csv_path}")

    # Cerchiamo la cartella specifica delle immagini
    if "224x224" in dirs:
        img_dir = os.path.join(root, "224x224")
        print(f"   -> Cartella Immagini Trovata: {img_dir}")

# --- VERIFICA E CREAZIONE DATASET ---
if csv_path and img_dir:
    print("\n✅ File trovati! Creazione Dataset in corso...")

    # Creiamo il dataset
    train_dataset = BookCoverDataset(
        csv_file=csv_path,
        root_dir=img_dir,
        transform=data_transforms['train']
    )

    # TEST RAPIDO
    print(f"Dataset caricato correttamente con {len(train_dataset)} libri.")
    print(f"Numero di classi (Generi): {len(train_dataset.classes)}")

    # Verifica GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n🖥️ Device attivo: {device}")
    if device.type == 'cuda':
        print("🚀 Perfetto! La GPU NVIDIA T4 è pronta a spingere.")
    else:
        print("⚠️ ATTENZIONE: Stai usando la CPU. Vai su Modifica -> Impostazioni Notebook -> Hardware Accelerator -> T4 GPU")

    # Test estrazione di un elemento
    img, label = train_dataset[0]
    print(f"\nTest Shape Tensore: {img.shape} (Deve essere 3, 224, 224)")

else:
    print("\n❌ ERRORE CRITICO: Non ho trovato i file necessari.")
    print("Controlla il contenuto dello zip. Cerco 'book30-listing-train.csv' e una cartella '224x224'.")

🔍 Scansione cartelle in corso...
   -> CSV Trovato: /content/dataset_unzipped/dataset/book30-listing-train.csv
   -> Cartella Immagini Trovata: /content/dataset_unzipped/dataset/title30cat/224x224

✅ File trovati! Creazione Dataset in corso...
Dataset caricato correttamente con 51300 libri.
Numero di classi (Generi): 30

🖥️ Device attivo: cuda
🚀 Perfetto! La GPU NVIDIA T4 è pronta a spingere.

Test Shape Tensore: torch.Size([3, 224, 224]) (Deve essere 3, 224, 224)


In [ ]:
###############CELLA 4##############
from torch.utils.data import DataLoader, random_split

# 1. Divisione Train / Validation (80% / 20%)
total_size = len(train_dataset)
train_len = int(0.8 * total_size)
val_len = total_size - train_len

# random_split mischia gli indici e crea due sotto-dataset
train_subset, val_subset = random_split(train_dataset, [train_len, val_len])

print(f"📊 Split completato:")
print(f"   -> Training Set: {len(train_subset)} immagini")
print(f"   -> Validation Set: {len(val_subset)} immagini")

# 2. Creazione dei DataLoader (I 'nastri trasportatori' per la GPU)
# Batch Size 64 è ottimale per la GPU T4 di Colab (usa bene la memoria senza esaurirla)
BATCH_SIZE = 64

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"✅ Dataloaders pronti (Batch size: {BATCH_SIZE})")

📊 Split completato:
   -> Training Set: 41040 immagini
   -> Validation Set: 10260 immagini
✅ Dataloaders pronti (Batch size: 64)


In [ ]:
###############CELLA 5###################
from torchvision import models
import torch.nn as nn

def get_model(num_classes=30):
    print("Scaricamento pesi ResNet50 (ImageNet)...")
    # Scarichiamo la versione più aggiornata dei pesi
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    # 1. FREEZING: Congeliamo tutti i parametri
    # Questo impedisce che durante il training modifichiamo i filtri che sanno già "vedere"
    for param in model.parameters():
        param.requires_grad = False

    # 2. Sostituzione dell'ultimo layer (Fully Connected)
    # ResNet50 ha 2048 feature in ingresso all'ultimo layer
    num_ftrs = model.fc.in_features

    # Creiamo il nuovo layer classificatore.
    # Nota: Di default, i nuovi layer hanno requires_grad=True, quindi QUESTI verranno addestrati.
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512), # Strato intermedio per imparare combinazioni complesse
        nn.ReLU(),
        nn.Dropout(0.5),          # Dropout per evitare overfitting (tecnica standard)
        nn.Linear(512, num_classes) # Output finale: 30 probabilità
    )

    return model

# Istanziamo il modello e lo spostiamo sulla GPU
model = get_model(num_classes=len(train_dataset.classes))
model = model.to(device) # Sposta tutto sulla GPU T4

print("\n🤖 Modello ResNet50 caricato e modificato per 30 classi.")
print("   -> Backbone (corpo): Congelato ❄️")
print("   -> Head (testa): Pronta per l'addestramento 🔥")

Scaricamento pesi ResNet50 (ImageNet)...

🤖 Modello ResNet50 caricato e modificato per 30 classi.
   -> Backbone (corpo): Congelato ❄️
   -> Head (testa): Pronta per l'addestramento 🔥


In [ ]:
##################CELLA 6#################
import torch.optim as optim
import time

# --- CONFIGURAZIONE TRAINING ---
# 1. Loss Function: CrossEntropy è standard per classificazione multi-classe
criterion = nn.CrossEntropyLoss()

# 2. Optimizer: Adam è solitamente più veloce a convergere rispetto a SGD
# Passiamo solo model.fc.parameters() perché il resto della rete è congelato!
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 3. Numero di Epoche (Quante volte la rete vede tutto il dataset)
# Inizia con 5 epoche per vedere se funziona, poi puoi aumentare a 10 o 20
NUM_EPOCHS = 5

# --- FUNZIONE DI TRAINING ---
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    start_time = time.time()

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Ogni epoca ha una fase di training e una di validation
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Mette il modello in modalità addestramento
                dataloader = train_loader
            else:
                model.eval()   # Mette il modello in modalità valutazione (congela dropout, ecc.)
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            # Iterazione sui batch
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Azzera i gradienti
                optimizer.zero_grad()

                # Forward (calcolo predizioni)
                # track_grad solo se siamo in training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + Optimize solo in training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistiche
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects.double() / total_samples

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    time_elapsed = time.time() - start_time
    print(f'\nAddestramento completato in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

# --- AVVIO TRAINING ---
# Salviamo il modello addestrato nella variabile 'trained_model'
print("🚀 Inizio Addestramento...")
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=NUM_EPOCHS)

🚀 Inizio Addestramento...

Epoch 1/5
----------
train Loss: 2.9255 Acc: 0.1937
val Loss: 2.7283 Acc: 0.2459

Epoch 2/5
----------
train Loss: 2.7430 Acc: 0.2343
val Loss: 2.6839 Acc: 0.2508

Epoch 3/5
----------
train Loss: 2.6790 Acc: 0.2511
val Loss: 2.6532 Acc: 0.2644

Epoch 4/5
----------
train Loss: 2.6259 Acc: 0.2630
val Loss: 2.6454 Acc: 0.2668

Epoch 5/5
----------
train Loss: 2.5847 Acc: 0.2738
val Loss: 2.6258 Acc: 0.2656

Addestramento completato in 23m 39s


In [ ]:
##############CELLA 7###############
# --- FINE TUNING ---

print("❄️ Scongelamento dei pesi del Backbone...")
# 1. Sblocchiamo TUTTI i parametri della rete
for param in trained_model.parameters():
    param.requires_grad = True

# 2. Nuovo Optimizer con Learning Rate MOLTO più basso
# Usiamo 1e-4 (0.0001) o 1e-5. Se è troppo alto, distruggiamo le conoscenze pregresse.
# Aggiungiamo weight_decay=1e-4 (o 1e-5)
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

# 3. Addestriamo per altre epoche (Fine-Tuning)
print("🚀 Inizio Fine-Tuning (tutta la rete)...")
finetuned_model = train_model(
    trained_model,
    train_loader,
    val_loader,
    criterion,
    optimizer_ft,
    num_epochs=10 # Proviamo 10 epoche, ci metterà un po' di più
)

❄️ Scongelamento dei pesi del Backbone...
🚀 Inizio Fine-Tuning (tutta la rete)...

Epoch 1/10
----------
train Loss: 2.4366 Acc: 0.3097
val Loss: 2.4745 Acc: 0.3048

Epoch 2/10
----------
train Loss: 2.2381 Acc: 0.3582
val Loss: 2.4444 Acc: 0.3186

Epoch 3/10
----------
train Loss: 2.0616 Acc: 0.4035
val Loss: 2.4568 Acc: 0.3175

Epoch 4/10
----------
train Loss: 1.8960 Acc: 0.4415
val Loss: 2.4640 Acc: 0.3139

Epoch 5/10
----------


KeyboardInterrupt: 